# File to analyze graphics of default simulation

## Imports to collect data and plot graphics

In [1]:
from components.graphic_generator import Graphic_Gen
from components.data_manager import Data_Manager
from components.utils.enums import Topologies
import components.network_data as nwd
from components.utils.enums import Colors
import numpy as np
import pandas as pd

## Constants

In [2]:
PATH_TO_CSV: str = "data/topology_simulation/topology_simulation.csv"
NUMBER_OF_NODES: tuple = tuple([12*i for i in range(1, 108, 12)])
PARAMS_TUPLE: tuple[float, float, float] = (0.1, 0.3, 0.5)
COLORS_LIST: tuple[Colors, Colors, Colors] = (Colors.YELLOW, Colors.ORANGE, Colors.RED)

## Load csv

In [3]:
dm: Data_Manager = Data_Manager()
loaded: bool = dm.load_csv(PATH_TO_CSV)
if not loaded:
    print("File don't loaded")

## Convert csv to pandas DataFrame

In [4]:
dframe: pd.DataFrame = dm.csv_to_dataframe()

## Create a function to collect default networks simulations (topologies: Barabási-Albert and Erdős-Rényi)

In [19]:
def collect_default_series_dict(topology: str, parameters_columns: list[str], desired_column: str, multiply_factor: int = 1, divisor_column: str | None = None) -> list[pd.Series]:
    dict_of_series: dict[pd.Series] = dict()
    
    for parameter in parameters_columns:
        mask: pd.Series = (
            (dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0) & 
            (dframe[nwd.TOPOLOGY] == topology) & 
            (dframe[nwd.PARAMETER] == parameter)
        )

        if divisor_column is None:
            dict_of_series[parameter] = (dframe.loc[mask, desired_column] * multiply_factor)
        else:
            dict_of_series[parameter] = (dframe.loc[mask, desired_column] / dframe.loc[mask, divisor_column]) * multiply_factor

    return dict_of_series

## Manipulate data to collect columns from default networks (without black holes)

In [ ]:
# Grid

default_grid_consummed_eprs_series: pd.Series = (dframe.loc[
    (dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0) & 
    (dframe[nwd.TOPOLOGY] == 'Grid'), 
    nwd.CONSUMED_EPRS]
)

# (total success / total requests) * 100
default_grid_success_tax_series: pd.Series = (dframe.loc[
    (dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0) & 
    (dframe[nwd.TOPOLOGY] == 'Grid'), 
    nwd.TOTAL_REQUEST_SUCCESS]
) / (dframe.loc[
    (dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0) & 
    (dframe[nwd.TOPOLOGY] == 'Grid'), 
    nwd.REQUESTS]
) * 100

# total fidelity / success requests
default_grid_avg_fidelity_series: pd.Series = (dframe.loc[
    (dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0) & 
    (dframe[nwd.TOPOLOGY] == 'Grid'), 
    nwd.TOTAL_ROUTE_FIDELITY]
) / (dframe.loc[
        (dframe[nwd.NUMBER_OF_BLACK_HOLES] == 0) & 
        (dframe[nwd.TOPOLOGY] == 'Grid'), 
        nwd.TOTAL_ROUTE_FIDELITY]
)


# Barabási-Albert

default_barabasi_albert_consummed_eprs_dict: dict[str, pd.Series] = collect_default_series_dict(
    topology='Barabási-Albert',
    parameters_columns=[f'm={i}' for i in range(1, 7, 2)],
    desired_column=nwd.CONSUMED_EPRS
)

# (total success / total requests) * 100
default_barabasi_albert_success_tax_dict: dict[str, pd.Series] = collect_default_series_dict(
    topology='Barabási-Albert',
    parameters_columns=[f'm={i}' for i in range(1, 7, 2)],
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    divisor_column=nwd.REQUESTS,
    multiply_factor=100
)

# total fidelity / success requests
default_barabasi_avg_fidelity_eprs_dict: dict[str, pd.Series] = collect_default_series_dict(
    topology='Barabási-Albert',
    parameters_columns=[f'm={i}' for i in range(1, 7, 2)],
    desired_column=nwd.TOTAL_ROUTE_FIDELITY,
    divisor_column=nwd.TOTAL_REQUEST_SUCCESS
)


# Erdős-Rényi

default_erdos_renyi_consummed_eprs_dict: dict[str, pd.Series] = collect_default_series_dict(
    topology='Erdős-Rényi',
    parameters_columns=[f'p={i}' for i in PARAMS_TUPLE],
    desired_column=nwd.CONSUMED_EPRS
)

# (total success / total requests) * 100
default_erdos_renyi_success_tax_dict: dict[str, pd.Series] = collect_default_series_dict(
    topology='Erdős-Rényi',
    parameters_columns=[f'p={i}' for i in PARAMS_TUPLE],
    desired_column=nwd.TOTAL_REQUEST_SUCCESS,
    divisor_column=nwd.REQUESTS,
    multiply_factor=100
)

# total fidelity / success requests
default_erdos_renyi_avg_fidelity_dict: dict[str, pd.Series] = collect_default_series_dict(
    topology='Erdős-Rényi',
    parameters_columns=[f'p={i}' for i in PARAMS_TUPLE],
    desired_column=nwd.TOTAL_ROUTE_FIDELITY,
    divisor_column=nwd.TOTAL_REQUEST_SUCCESS
)